<a href="https://colab.research.google.com/github/Viny2030/tensorflow/blob/main/1bert_preprocessing_guide.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# BERT Preprocessing with TF Text

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/text/guide/bert_preprocessing_guide"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/text/blob/master/docs/guide/bert_preprocessing_guide.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/text/blob/master/docs/guide/bert_preprocessing_guide.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/text/docs/guide/bert_preprocessing_guide.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

## Overview

Text preprocessing is the end-to-end transformation of raw text into a model’s integer inputs. NLP models are often accompanied by several hundreds (if not thousands) of lines of Python code for preprocessing text. Text preprocessing is often a challenge for models because:

* **Training-serving skew.** It becomes increasingly difficult to ensure that the preprocessing logic of the model's inputs are consistent at all stages of model development (e.g. pretraining, fine-tuning, evaluation, inference).
Using different hyperparameters, tokenization, string preprocessing algorithms or simply packaging model inputs inconsistently at different stages could yield hard-to-debug and disastrous effects to the model.

* **Efficiency and flexibility.** While preprocessing can be done offline (e.g. by writing out processed outputs to files on disk and then reconsuming said preprocessed data in the input pipeline), this method incurs an additional file read and write cost. Preprocessing offline is also inconvenient if there are preprocessing decisions that need to happen dynamically. Experimenting with a different option would require regenerating the dataset again.

* **Complex model interface.** Text models are much more understandable when their inputs are pure text. It's hard to understand a model when its inputs require an extra, indirect encoding step. Reducing the preprocessing complexity is especially appreciated for model debugging, serving, and evaluation.

Additionally, simpler model interfaces also make it more convenient to try the model (e.g. inference or training) on different, unexplored datasets.


## Text preprocessing with TF.Text

Using TF.Text's text preprocessing APIs, we can construct a preprocessing
function that can transform a user's text dataset into the model's
integer inputs. Users can package preprocessing directly as part of their model to alleviate the above mentioned problems.

This tutorial will show how to use TF.Text preprocessing ops to transform text data into inputs for the BERT model and inputs for language masking pretraining task described in "Masked LM and Masking Procedure" of [BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding](https://arxiv.org/pdf/1810.04805.pdf). The process involves tokenizing text into subword units, combining sentences, trimming content to a fixed size and extracting labels for the masked language modeling task.

# **Preprocesamiento de texto con TF.Text**
Usando las API de preprocesamiento de texto de TF.Text, podemos construir una función de preprocesamiento que puede transformar el conjunto de datos de texto de un usuario en las entradas de números enteros del modelo. Los usuarios pueden empaquetar el preprocesamiento directamente como parte de su modelo para aliviar los problemas mencionados anteriormente.

Este tutorial mostrará cómo usar las operaciones de preprocesamiento de TF.Text para transformar datos de texto en entradas para el modelo BERT y entradas para la tarea de preentrenamiento de enmascaramiento de lenguaje descrita en "LM enmascarado y procedimiento de enmascaramiento" de BERT: preentrenamiento de transformadores bidireccionales profundos para la comprensión del lenguaje. El proceso implica convertir el texto en unidades de subpalabras, combinar oraciones, recortar el contenido a un tamaño fijo y extraer etiquetas para la tarea de modelado de lenguaje enmascarado.


### Setup

Let's import the packages and libraries we need first.

In [ ]:
!pip install -q -U "tensorflow-text==2.11.*"

In [ ]:
import tensorflow as tf
import tensorflow_text as text
import functools
print("TensorFlow version: ", tf.__version__)

TensorFlow version:  2.11.1


Our data contains two text features and we can create a example `tf.data.Dataset`. Our goal is to create a function that we can supply `Dataset.map()` with to be used in training.

Nuestros datos contienen dos características de texto y podemos crear un ejemplo tf.data.Dataset. Nuestro objetivo es crear una función que podamos suministrar a Dataset.map() para que se use en el entrenamiento.

In [ ]:
examples = {
    "text_a": [
      "Sponge bob Squarepants is an Avenger",
      "Marvel Avengers"
    ],
    "text_b": [
     "Barack Obama is the President.",
     "President is the highest office"
  ],
}

dataset = tf.data.Dataset.from_tensor_slices(examples)
next(iter(dataset))

{'text_a': <tf.Tensor: shape=(), dtype=string, numpy=b'Sponge bob Squarepants is an Avenger'>,
 'text_b': <tf.Tensor: shape=(), dtype=string, numpy=b'Barack Obama is the President.'>}

In [ ]:
examples1 = {
    "text_a": [
      "Jane bought me these books",
       "Jane bought a book for me",
       "She dropped a line to him",
       "She sleeps",
        "I sleep a lot",
      "I was born in Madrid"
    ],
    "text_b": [
"the cat was chased by the dog",
"I was born in Madrid during 1995",
"Out of all this,something good will come",
"Susan left after the rehearsal. She did it well",
"She sleeps during the morning",
"but she sleeps"
  ],
}
dataset1 = tf.data.Dataset.from_tensor_slices(examples1)
next(iter(dataset1))

{'text_a': <tf.Tensor: shape=(), dtype=string, numpy=b'Jane bought me these books'>,
 'text_b': <tf.Tensor: shape=(), dtype=string, numpy=b'the cat was chased by the dog'>}

### Tokenizing

Our first step is to run any string preprocessing and tokenize our dataset. This can be done using the [`text.BertTokenizer`](https://tensorflow.org/text/api_docs/python/text/BertTokenizer), which is a [`text.Splitter`](https://tensorflow.org/text/api_docs/python/text/Splitter) that can tokenize sentences into subwords or wordpieces for the [BERT model](https://github.com/google-research/bert) given a vocabulary generated from the [Wordpiece algorithm](https://www.tensorflow.org/text/guide/subwords_tokenizer#optional_the_algorithm). You can learn more about other subword tokenizers available in TF.Text from [here](https://www.tensorflow.org/text/guide/subwords_tokenizer).


The vocabulary can be from a previously generated BERT checkpoint, or you can generate one yourself on your own data. For the purposes of this example, let's create a toy vocabulary:

# Tokenización
Nuestro primer paso es ejecutar cualquier preprocesamiento de cadenas y tokenizar nuestro conjunto de datos. Esto se puede hacer usando text.BertTokenizer, que es un text.Splitter que puede tokenizar oraciones en subpalabras o fragmentos de palabras para el modelo BERT dado un vocabulario generado a partir del algoritmo Wordpiece. Puede obtener más información sobre otros tokenizadores de subpalabras disponibles en TF.Text aquí.

El vocabulario puede provenir de un punto de control BERT generado previamente, o puede generar uno usted mismo con sus propios datos. Para los fines de este ejemplo, creemos un vocabulario de juguete:

In [ ]:
_VOCAB = [
    # Special tokens
    b"[UNK]", b"[MASK]", b"[RANDOM]", b"[CLS]", b"[SEP]",
    # Suffixes
    b"##ack", b"##ama", b"##ger", b"##gers", b"##onge", b"##pants",  b"##uare",
    b"##vel", b"##ven", b"an", b"A", b"Bar", b"Hates", b"Mar", b"Ob",
    b"Patrick", b"President", b"Sp", b"Sq", b"bob", b"box", b"has", b"highest",
    b"is", b"office", b"the",
]

_START_TOKEN = _VOCAB.index(b"[CLS]")
_END_TOKEN = _VOCAB.index(b"[SEP]")
_MASK_TOKEN = _VOCAB.index(b"[MASK]")
_RANDOM_TOKEN = _VOCAB.index(b"[RANDOM]")
_UNK_TOKEN = _VOCAB.index(b"[UNK]")
_MAX_SEQ_LEN = 8
_MAX_PREDICTIONS_PER_BATCH = 5

_VOCAB_SIZE = len(_VOCAB)

lookup_table = tf.lookup.StaticVocabularyTable(
    tf.lookup.KeyValueTensorInitializer(
      keys=_VOCAB,
      key_dtype=tf.string,
      values=tf.range(
          tf.size(_VOCAB, out_type=tf.int64), dtype=tf.int64),
          value_dtype=tf.int64
        ),
      num_oov_buckets=1
)

Let's construct a [`text.BertTokenizer`](https://tensorflow.org/text/api_docs/python/text/BertTokenizer) using the above vocabulary and tokenize the text inputs into a [`RaggedTensor`](https://www.tensorflow.org/api_docs/python/tf/RaggedTensor).`.

Construyamos un text.BertTokenizer usando el vocabulario anterior y tokenicemos las entradas de texto en un RaggedTensor.`.

In [ ]:
bert_tokenizer = text.BertTokenizer(lookup_table, token_out_type=tf.string)
bert_tokenizer.tokenize(examples["text_a"])

<tf.RaggedTensor [[[b'Sp', b'##onge'], [b'bob'], [b'Sq', b'##uare', b'##pants'], [b'is'],
  [b'an'], [b'A', b'##ven', b'##ger']]                                  ,
 [[b'Mar', b'##vel'], [b'A', b'##ven', b'##gers']]]>

In [ ]:
bert_tokenizer1 = text.BertTokenizer(lookup_table, token_out_type=tf.string)
bert_tokenizer1.tokenize(examples1["text_a"])

<tf.RaggedTensor [[[b'[UNK]'],
  [b'[UNK]'],
  [b'[UNK]'],
  [b'[UNK]'],
  [b'[UNK]']], [[b'[UNK]'],
                [b'[UNK]'],
                [b'[UNK]'],
                [b'[UNK]'],
                [b'[UNK]'],
                [b'[UNK]']], [[b'[UNK]'],
                              [b'[UNK]'],
                              [b'[UNK]'],
                              [b'[UNK]'],
                              [b'[UNK]'],
                              [b'[UNK]']], [[b'[UNK]'],
                                            [b'[UNK]']], [[b'[UNK]'],
                                                          [b'[UNK]'],
                                                          [b'[UNK]'],
                                                          [b'[UNK]']],
 [[b'[UNK]'],
  [b'[UNK]'],
  [b'[UNK]'],
  [b'[UNK]'],
  [b'[UNK]']]]>

In [ ]:
bert_tokenizer1.tokenize(examples1["text_b"])

<tf.RaggedTensor [[[b'the'],
  [b'[UNK]'],
  [b'[UNK]'],
  [b'[UNK]'],
  [b'[UNK]'],
  [b'the'],
  [b'[UNK]']], [[b'[UNK]'],
                [b'[UNK]'],
                [b'[UNK]'],
                [b'[UNK]'],
                [b'[UNK]'],
                [b'[UNK]'],
                [b'[UNK]']], [[b'[UNK]'],
                              [b'[UNK]'],
                              [b'[UNK]'],
                              [b'[UNK]'],
                              [b'[UNK]'],
                              [b'[UNK]'],
                              [b'[UNK]'],
                              [b'[UNK]'],
                              [b'[UNK]']], [[b'[UNK]'],
                                            [b'[UNK]'],
                                            [b'[UNK]'],
                                            [b'the'],
                                            [b'[UNK]'],
                                            [b'[UNK]'],
                                            [b'[UNK]'],
         

Text output from [`text.BertTokenizer`](https://tensorflow.org/text/api_docs/python/text/BertTokenizer) allows us see how the text is being tokenized, but the model requires integer IDs. We can set the `token_out_type` param to `tf.int64` to obtain integer IDs (which are the indices into the vocabulary).

La salida de texto de text.BertTokenizer nos permite ver cómo se está tokenizando el texto, pero el modelo requiere identificadores enteros. Podemos establecer el parámetro token_out_type en tf.int64 para obtener identificadores enteros (que son los índices del vocabulario).

In [ ]:
bert_tokenizer1 = text.BertTokenizer(lookup_table, token_out_type=tf.int64)
segment_a1 = bert_tokenizer1.tokenize(examples["text_a"])
segment_a1

<tf.RaggedTensor [[[22, 9], [24], [23, 11, 10], [28], [14], [15, 13, 7]],
 [[18, 12], [15, 13, 8]]]>

In [ ]:
bert_tokenizer1 = text.BertTokenizer(lookup_table, token_out_type=tf.int64)
segment_a1 = bert_tokenizer1.tokenize(examples["text_a"])
segment_a1

<tf.RaggedTensor [[[22, 9], [24], [23, 11, 10], [28], [14], [15, 13, 7]],
 [[18, 12], [15, 13, 8]]]>

In [ ]:
segment_b1 = bert_tokenizer1.tokenize(examples["text_b"])
segment_b1

<tf.RaggedTensor [[[16, 5], [19, 6], [28], [30], [21], [0]], [[21], [28], [30], [27], [29]]]>

[`text.BertTokenizer`](https://tensorflow.org/text/api_docs/python/text/BertTokenizer) returns a `RaggedTensor` with shape `[batch, num_tokens, num_wordpieces]`. Because we don't need the extra `num_tokens` dimensions for our current use case,  we can merge the last two dimensions to obtain a `RaggedTensor` with shape `[batch, num_wordpieces]`:

text.BertTokenizer devuelve un RaggedTensor con forma [batch, num_tokens, num_wordpieces]. Como no necesitamos las dimensiones num_tokens adicionales para nuestro caso de uso actual, podemos fusionar las dos últimas dimensiones para obtener un RaggedTensor con forma [batch, num_wordpieces]:

In [ ]:
segment_a1 = segment_a1.merge_dims(-2, -1)
segment_a1

<tf.RaggedTensor [[22, 9, 24, 23, 11, 10, 28, 14, 15, 13, 7], [18, 12, 15, 13, 8]]>

In [ ]:
segment_b1 = segment_b1.merge_dims(-2, -1)
segment_b1

<tf.RaggedTensor [[16, 5, 19, 6, 28, 30, 21, 0], [21, 28, 30, 27, 29]]>

### Content Trimming

The main input to BERT is a concatenation of two sentences. However, BERT requires inputs to be in a fixed-size and shape and we may have content which exceed our budget.

We can tackle this by using a [`text.Trimmer`](https://tensorflow.org/text/api_docs/python/text/Trimmer) to trim our content down to a predetermined size (once concatenated along the last axis). There are different `text.Trimmer` types which select content to preserve using different algorithms. [`text.RoundRobinTrimmer`](https://tensorflow.org/text/api_docs/python/text/RoundRobinTrimmer) for example will allocate quota equally for each segment but may trim the ends of sentences. [`text.WaterfallTrimmer`](https://tensorflow.org/text/api_docs/python/text/WaterfallTrimmer) will trim starting from the end of the last sentence.

For our example, we will use `RoundRobinTrimmer` which selects items from each segment in a left-to-right manner.


# Recorte de contenido
La entrada principal de BERT es una concatenación de dos oraciones. Sin embargo, BERT requiere que las entradas tengan un tamaño y una forma fijos y es posible que tengamos contenido que exceda nuestro presupuesto.

Podemos solucionar esto utilizando un text.Trimmer para recortar nuestro contenido a un tamaño predeterminado (una vez concatenado a lo largo del último eje). Hay diferentes tipos de text.Trimmer que seleccionan el contenido que se conservará utilizando diferentes algoritmos. text.RoundRobinTrimmer, por ejemplo, asignará una cuota equitativa para cada segmento, pero puede recortar los finales de las oraciones. text.WaterfallTrimmer recortará comenzando desde el final de la última oración.

Para nuestro ejemplo, utilizaremos RoundRobinTrimmer, que selecciona elementos de cada segmento de izquierda a derecha.

In [ ]:
trimmer = text.RoundRobinTrimmer(max_seq_length=_MAX_SEQ_LEN)
trimmed = trimmer.trim([segment_a1, segment_b1])
trimmed

[<tf.RaggedTensor [[22, 9, 24, 23],
  [18, 12, 15, 13]]>,
 <tf.RaggedTensor [[16, 5, 19, 6],
  [21, 28, 30, 27]]>]

`trimmed` now contains the segments where the number of elements across a batch is 8 elements (when concatenated along axis=-1).

recortado ahora contiene los segmentos donde el número de elementos en un lote es de 8 elementos (cuando se concatenan a lo largo del eje = -1).

### Combining segments

Now that we have segments trimmed, we can combine them together to get a single `RaggedTensor`. BERT uses special tokens to indicate the beginning (`[CLS]`) and end of a segment (`[SEP]`). We also need a `RaggedTensor` indicating which items in the combined `Tensor` belong to which segment. We can use [`text.combine_segments()`](https://tensorflow.org/text/api_docs/python/text/combine_segments) to get both of these `Tensor` with special tokens inserted.

# Combinación de segmentos
Ahora que hemos recortado los segmentos, podemos combinarlos para obtener un único tensor irregular. BERT utiliza tokens especiales para indicar el comienzo ([CLS]) y el final de un segmento ([SEP]). También necesitamos un tensor irregular que indique qué elementos del tensor combinado pertenecen a qué segmento. Podemos utilizar text.combine_segments() para obtener ambos tensores con tokens especiales insertados.

In [ ]:
segments_combined, segments_ids = text.combine_segments(
  trimmed,
  start_of_sequence_id=_START_TOKEN, end_of_segment_id=_END_TOKEN)
segments_combined, segments_ids

(<tf.RaggedTensor [[3, 22, 9, 24, 23, 4, 16, 5, 19, 6, 4],
  [3, 18, 12, 15, 13, 4, 21, 28, 30, 27, 4]]>,
 <tf.RaggedTensor [[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
  [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]]>)

### Masked Language Model Task

Now that we have our basic inputs, we can begin to extract the inputs needed for the "Masked LM and Masking Procedure" task described in [BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding](https://arxiv.org/pdf/1810.04805.pdf)

The masked language model task has two sub-problems for us to think about: (1) what items to select for masking and (2) what values are they assigned?


# **Tarea de modelo de lenguaje enmascarado**
Ahora que tenemos nuestras entradas básicas, podemos comenzar a extraer las entradas necesarias para la tarea "LM enmascarado y procedimiento de enmascaramiento" descrita en BERT: preentrenamiento de transformadores bidireccionales profundos para la comprensión del lenguaje

La tarea de modelo de lenguaje enmascarado tiene dos subproblemas en los que debemos pensar: (1) qué elementos seleccionar para enmascarar y (2) ¿qué valores se les asignan?


#### Item Selection
Because we will choose to select items randomly for masking, we will use a [`text.RandomItemSelector`](https://tensorflow.org/text/api_docs/python/text/RandomItemSelector). `RandomItemSelector` randomly selects items in a batch subject to restrictions given (`max_selections_per_batch`, `selection_rate` and `unselectable_ids`) and returns a boolean mask indicating which items were selected.

# Selección de elementos
Como elegiremos seleccionar elementos al azar para enmascararlos, utilizaremos un text.RandomItemSelector. RandomItemSelector selecciona elementos al azar en un lote sujeto a restricciones dadas (max_selections_per_batch, selection_rate y unselectable_ids) y devuelve una máscara booleana que indica qué elementos fueron seleccionados

In [ ]:
random_selector = text.RandomItemSelector(
    max_selections_per_batch=_MAX_PREDICTIONS_PER_BATCH,
    selection_rate=0.2,
    unselectable_ids=[_START_TOKEN, _END_TOKEN, _UNK_TOKEN]
)
selected = random_selector.get_selection_mask(
    segments_combined, axis=1)
selected

<tf.RaggedTensor [[False, False, False, False, True, False, True, False, False, False,
  False],
 [False, True, False, False, False, False, True, False, False, False,
  False]]>

#### Choosing the Masked Value

The methodology described the original BERT paper for choosing the value for masking is as follows:

For `mask_token_rate` of the time, replace the item with the `[MASK]` token:

    "my dog is hairy" -> "my dog is [MASK]"

For `random_token_rate` of the time, replace the item with a random word:

    "my dog is hairy" -> "my dog is apple"

For `1 - mask_token_rate - random_token_rate` of the time, keep the item
unchanged:

    "my dog is hairy" -> "my dog is hairy."

[`text.MaskedValuesChooser`](https://tensorflow.org/text/api_docs/python/text/MaskValuesChooser) encapsulates this logic and can be used for our preprocessing function. Here's an example of what `MaskValuesChooser` returns given a `mask_token_rate` of 80% and default `random_token_rate`:


# Elección del valor enmascarado
La metodología descrita en el artículo original de BERT para elegir el valor de enmascaramiento es la siguiente:

Para mask_token_rate del tiempo, reemplace el elemento con el token [MASK]:

"mi perro es peludo" -> "mi perro es [MASK]"
Para random_token_rate del tiempo, reemplace el elemento con una palabra aleatoria:

"mi perro es peludo" -> "mi perro es manzana"
Para 1 - mask_token_rate - random_token_rate del tiempo, mantenga el elemento sin cambios:

"mi perro es peludo" -> "mi perro es peludo".
text.MaskedValuesChooser encapsula esta lógica y se puede utilizar para nuestra función de preprocesamiento. Aquí hay un ejemplo de lo que MaskValuesChooser devuelve dado un mask_token_rate del 80% y un random_token_rate predeterminado:


In [ ]:
mask_values_chooser = text.MaskValuesChooser(_VOCAB_SIZE, _MASK_TOKEN, 0.8)
mask_values_chooser.get_mask_values(segments_combined)

<tf.RaggedTensor [[1, 1, 1, 1, 23, 1, 1, 1, 1, 1, 23],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]>

When supplied with a `RaggedTensor` input, `text.MaskValuesChooser` returns a `RaggedTensor` of the same shape with either `_MASK_VALUE` (0), a random ID, or the same unchanged id.

Cuando se suministra con una entrada RaggedTensor, text.MaskValuesChooser devuelve un RaggedTensor de la misma forma con _MASK_VALUE (0), una ID aleatoria o la misma ID sin cambios.

#### Generating Inputs for Masked Language Model Task

Now that we have a `RandomItemSelector` to help us select items for masking and `text.MaskValuesChooser` to assign the values, we can use [`text.mask_language_model()`](https://tensorflow.org/text/api_docs/python/text/mask_language_model) to assemble all the inputs of this task for our BERT model.


Generación de entradas para la tarea del modelo de lenguaje enmascarado
Ahora que tenemos un RandomItemSelector para ayudarnos a seleccionar elementos para enmascarar y un text.MaskValuesChooser para asignar los valores, podemos usar text.mask_language_model() para ensamblar todas las entradas de esta tarea para nuestro modelo BERT.

In [ ]:
masked_token_ids, masked_pos, masked_lm_ids = text.mask_language_model(
  segments_combined,
  item_selector=random_selector, mask_values_chooser=mask_values_chooser)

Let's dive deeper and examine the outputs of `mask_language_model()`. The output of `masked_token_ids` is:

Profundicemos más y examinemos los resultados de mask_language_model(). El resultado de masked_token_ids es:

In [ ]:
masked_token_ids

<tf.RaggedTensor [[3, 1, 1, 24, 23, 4, 16, 5, 19, 6, 4],
 [3, 1, 1, 15, 13, 4, 21, 28, 30, 27, 4]]>

Remember that our input is encoded using a vocabulary. If we decode `masked_token_ids` using our vocabulary, we get:

Recuerde que nuestra entrada está codificada mediante un vocabulario. Si decodificamos masked_token_ids mediante nuestro vocabulario, obtenemos:

In [ ]:
tf.gather(_VOCAB, masked_token_ids)

<tf.RaggedTensor [[b'[CLS]', b'[MASK]', b'[MASK]', b'bob', b'Sq', b'[SEP]', b'Bar',
  b'##ack', b'Ob', b'##ama', b'[SEP]'],
 [b'[CLS]', b'[MASK]', b'[MASK]', b'A', b'##ven', b'[SEP]', b'President',
  b'is', b'the', b'highest', b'[SEP]']]>

Notice that some wordpiece tokens have been replaced with either `[MASK]`, `[RANDOM]` or a different ID value. `masked_pos` output gives us the indices (in the respective batch) of the tokens that have been replaced.

Tenga en cuenta que algunos tokens de piezas de palabras han sido reemplazados con [MASK], [RANDOM] o un valor de identificación diferente. La salida de masked_pos nos da los índices (en el lote respectivo) de los tokens que han sido reemplazados.

In [ ]:
masked_pos

<tf.RaggedTensor [[1, 2],
 [1, 2]]>

`masked_lm_ids` gives us the original value of the token.

masked_lm_ids nos da el valor original del token.

In [ ]:
masked_lm_ids

<tf.RaggedTensor [[22, 9],
 [18, 12]]>

We can again decode the IDs here to get human readable values.


Podemos nuevamente decodificar los ID aquí para obtener valores legibles para humanos.

In [ ]:
tf.gather(_VOCAB, masked_lm_ids)

<tf.RaggedTensor [[b'Sp', b'##onge'],
 [b'Mar', b'##vel']]>

### Padding Model Inputs

Now that we have all the inputs for our model, the last step in our preprocessing is to package them into fixed 2-dimensional `Tensor`s with padding and also generate a mask `Tensor` indicating the values which are pad values. We can use [`text.pad_model_inputs()`](https://tensorflow.org/text/api_docs/python/text/pad_model_inputs) to help us with this task.

# Entradas del modelo con relleno
Ahora que tenemos todas las entradas para nuestro modelo, el último paso en nuestro preprocesamiento es empaquetarlas en tensores fijos bidimensionales con relleno y también generar un tensor de máscara que indique los valores que son valores de relleno. Podemos usar text.pad_model_inputs() para ayudarnos con esta tarea.

In [ ]:
# Prepare and pad combined segment inputs
input_word_ids, input_mask = text.pad_model_inputs(
  masked_token_ids, max_seq_length=_MAX_SEQ_LEN)
input_type_ids, _ = text.pad_model_inputs(
  segments_ids, max_seq_length=_MAX_SEQ_LEN)

# Prepare and pad masking task inputs
masked_lm_positions, masked_lm_weights = text.pad_model_inputs(
  masked_pos, max_seq_length=_MAX_PREDICTIONS_PER_BATCH)
masked_lm_ids, _ = text.pad_model_inputs(
  masked_lm_ids, max_seq_length=_MAX_PREDICTIONS_PER_BATCH)

model_inputs = {
    "input_word_ids": input_word_ids,
    "input_mask": input_mask,
    "input_type_ids": input_type_ids,
    "masked_lm_ids": masked_lm_ids,
    "masked_lm_positions": masked_lm_positions,
    "masked_lm_weights": masked_lm_weights,
}
model_inputs

{'input_word_ids': <tf.Tensor: shape=(2, 8), dtype=int64, numpy=
 array([[ 3,  1,  1, 24, 23,  4, 16,  5],
        [ 3,  1,  1, 15, 13,  4, 21, 28]])>,
 'input_mask': <tf.Tensor: shape=(2, 8), dtype=int64, numpy=
 array([[1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1]])>,
 'input_type_ids': <tf.Tensor: shape=(2, 8), dtype=int64, numpy=
 array([[0, 0, 0, 0, 0, 0, 1, 1],
        [0, 0, 0, 0, 0, 0, 1, 1]])>,
 'masked_lm_ids': <tf.Tensor: shape=(2, 5), dtype=int64, numpy=
 array([[22,  9,  0,  0,  0],
        [18, 12,  0,  0,  0]])>,
 'masked_lm_positions': <tf.Tensor: shape=(2, 5), dtype=int64, numpy=
 array([[1, 2, 0, 0, 0],
        [1, 2, 0, 0, 0]])>,
 'masked_lm_weights': <tf.Tensor: shape=(2, 5), dtype=int64, numpy=
 array([[1, 1, 0, 0, 0],
        [1, 1, 0, 0, 0]])>}

## Review

Let's review what we have so far and assemble our preprocessing function. Here's what we have:

Revisión
Repasemos lo que tenemos hasta ahora y ensamblemos nuestra función de preprocesamiento. Esto es lo que tenemos:

In [ ]:
def bert_pretrain_preprocess(vocab_table, features):
  # Input is a string Tensor of documents, shape [batch, 1].
  text_a = features["text_a"]
  text_b = features["text_b"]

  # Tokenize segments to shape [num_sentences, (num_words)] each.
  tokenizer = text.BertTokenizer(
      vocab_table,
      token_out_type=tf.int64)
  segments = [tokenizer.tokenize(text).merge_dims(
      1, -1) for text in (text_a, text_b)]

  # Truncate inputs to a maximum length.
  trimmer = text.RoundRobinTrimmer(max_seq_length=6)
  trimmed_segments = trimmer.trim(segments)

  # Combine segments, get segment ids and add special tokens.
  segments_combined, segment_ids = text.combine_segments(
      trimmed_segments,
      start_of_sequence_id=_START_TOKEN,
      end_of_segment_id=_END_TOKEN)

  # Apply dynamic masking task.
  masked_input_ids, masked_lm_positions, masked_lm_ids = (
      text.mask_language_model(
        segments_combined,
        random_selector,
        mask_values_chooser,
      )
  )

  # Prepare and pad combined segment inputs
  input_word_ids, input_mask = text.pad_model_inputs(
    masked_input_ids, max_seq_length=_MAX_SEQ_LEN)
  input_type_ids, _ = text.pad_model_inputs(
    segment_ids, max_seq_length=_MAX_SEQ_LEN)

  # Prepare and pad masking task inputs
  masked_lm_positions, masked_lm_weights = text.pad_model_inputs(
    masked_lm_positions, max_seq_length=_MAX_PREDICTIONS_PER_BATCH)
  masked_lm_ids, _ = text.pad_model_inputs(
    masked_lm_ids, max_seq_length=_MAX_PREDICTIONS_PER_BATCH)

  model_inputs = {
      "input_word_ids": input_word_ids,
      "input_mask": input_mask,
      "input_type_ids": input_type_ids,
      "masked_lm_ids": masked_lm_ids,
      "masked_lm_positions": masked_lm_positions,
      "masked_lm_weights": masked_lm_weights,
  }
  return model_inputs

We previously constructed a `tf.data.Dataset` and we can now use our assembled preprocessing function `bert_pretrain_preprocess()` in `Dataset.map()`. This allows us to create an input pipeline for transforming our raw string data into integer inputs and feed directly into our model.

Anteriormente construimos un tf.data.Dataset y ahora podemos usar nuestra función de preprocesamiento ensamblada bert_pretrain_preprocess() en Dataset.map(). Esto nos permite crear una secuencia de entrada para transformar nuestros datos de cadena sin procesar en entradas de números enteros y alimentarlos directamente a nuestro modelo.

In [ ]:
dataset = (
    tf.data.Dataset.from_tensors(examples)
    .map(functools.partial(bert_pretrain_preprocess, lookup_table))
)

next(iter(dataset))

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


{'input_word_ids': <tf.Tensor: shape=(2, 8), dtype=int64, numpy=
 array([[ 3, 22,  1, 24,  4, 16,  5, 19],
        [ 3, 18, 12,  1,  4, 21,  1, 30]])>,
 'input_mask': <tf.Tensor: shape=(2, 8), dtype=int64, numpy=
 array([[1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1]])>,
 'input_type_ids': <tf.Tensor: shape=(2, 8), dtype=int64, numpy=
 array([[0, 0, 0, 0, 0, 1, 1, 1],
        [0, 0, 0, 0, 0, 1, 1, 1]])>,
 'masked_lm_ids': <tf.Tensor: shape=(2, 5), dtype=int64, numpy=
 array([[ 9, 16,  0,  0,  0],
        [15, 28,  0,  0,  0]])>,
 'masked_lm_positions': <tf.Tensor: shape=(2, 5), dtype=int64, numpy=
 array([[2, 5, 0, 0, 0],
        [3, 6, 0, 0, 0]])>,
 'masked_lm_weights': <tf.Tensor: shape=(2, 5), dtype=int64, numpy=
 array([[1, 1, 0, 0, 0],
        [1, 1, 0, 0, 0]])>}

## Related Tutorials

* [Classify text with BERT](https://www.tensorflow.org/text/tutorials/classify_text_with_bert) - A tutorial on how to use a pretrained BERT model to classify text. This is a nice follow up now that you are familiar with how to preprocess the inputs used by the BERT model.

* [Tokenizing with TF Text](https://www.tensorflow.org/text/guide/tokenizers) - Tutorial detailing the different types of tokenizers that exist in TF.Text.

* [Handling Text with `RaggedTensor`](https://www.tensorflow.org/guide/ragged_tensor) - Detailed guide on how to create, use and manipulate `RaggedTensor`s.
